# NoETL Playbook Execution Report
__Agent 007 mission report:__ _Operation completed. Martini status: shaken, not stirred._
- This notebook provides a report of the execution of a NoETL playbook using the `agent007.py` script.
- It analyzes the DuckDB database generated during the playbook run, with execution flow, step results, loop iterations, task executions, and context data captured throughout the process.
1. Run playbook with `agent007.py`.
2. Set `db_path` below to that file path.
3. Run all paragraphs to inspect tables and data.


In [1]:
import duckdb
import polars as pl
import os
import matplotlib.pyplot as plt
import pandas as pd
import json
import networkx as nx
from datetime import datetime

# Path to DuckDB database
db_path = '../noetl/runtime/agent007.duckdb'  # Default path used by agent007.py
# Example: db_path = '/Users/kadyapam/projects/noetl/noetl/noetl/runtime/agent007.duckdb'

if not os.path.exists(db_path) and db_path != ':memory:':
    raise FileNotFoundError(f"DuckDB database not found: {db_path}")

con = duckdb.connect(db_path)

def parse_json_column(df, column_name):
    if column_name in df.columns:
        def safe_json_parse(x):
            if not x:
                return None
            try:
                return json.loads(x)
            except json.JSONDecodeError as e:
                print(f"Warning: Could not parse JSON: {e}")
                return x

        return df.with_columns([
            pl.col(column_name).map_elements(safe_json_parse, return_dtype=pl.Object)
        ])
    return df

# execution ID
latest_execution = con.execute("""
    SELECT execution_id, MAX(timestamp) as latest_time
    FROM event_log
    GROUP BY execution_id
    ORDER BY latest_time DESC
    LIMIT 1
""").fetchone()

if latest_execution:
    execution_id = latest_execution[0]
    print(f"Analyzing execution ID: {execution_id}")
else:
    print("No executions found in the database.")
    execution_id = None


Analyzing execution ID: da456514-00e8-4e99-bb18-02d50ea3ec86


In [2]:
%load_ext sql
%sql con --alias duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

## List all tables


In [3]:
tables = con.execute("SHOW TABLES").fetchall()
for t in tables:
    print(t[0])


context
event_log
loop_state
step_results
task_results
transitions
workbook
workflow


## Execution Overview

This section provides a high-level overview of the playbook execution.


In [4]:
if execution_id:
    # execution start end times
    execution_times = con.execute("""
        SELECT 
            MIN(timestamp) as start_time,
            MAX(timestamp) as end_time,
            (EXTRACT(EPOCH FROM MAX(timestamp)) - EXTRACT(EPOCH FROM MIN(timestamp))) as duration_seconds
        FROM event_log
        WHERE execution_id = ?
    """, [execution_id]).fetchone()

    # execution statistics
    execution_stats = con.execute("""
        SELECT
            (SELECT COUNT(*) FROM step_results WHERE execution_id = ?) as step_count,
            (SELECT COUNT(*) FROM task_results WHERE execution_id = ?) as task_count,
            (SELECT COUNT(*) FROM loop_state WHERE execution_id = ?) as loop_count,
            (SELECT COUNT(*) FROM event_log WHERE execution_id = ? AND event_type LIKE '%error%') as error_count
    """, [execution_id, execution_id, execution_id, execution_id]).fetchone()

    # playbook name
    playbook_info = con.execute("""
        SELECT node_name, metadata
        FROM event_log
        WHERE execution_id = ? AND event_type = 'execution_start'
        LIMIT 1
    """, [execution_id]).fetchone()

    # execution overview
    print(f"Playbook: {playbook_info[0] if playbook_info else 'Unknown'}")
    print(f"Start Time: {execution_times[0]}")
    print(f"End Time: {execution_times[1]}")
    print(f"Duration: {execution_times[2]:.2f} seconds")
    print(f"Steps Executed: {execution_stats[0]}")
    print(f"Tasks Executed: {execution_stats[1]}")
    print(f"Loops Executed: {execution_stats[2]}")
    print(f"Errors Encountered: {execution_stats[3]}")

    # playbook metadata
    if playbook_info and playbook_info[1]:
        try:
            metadata = json.loads(playbook_info[1])
            print("\nPlaybook Metadata:")
            for key, value in metadata.items():
                print(f"  {key}: {value}")
        except:
            pass
else:
    print("No execution ID available.")


Playbook: weather_iterator_example
Start Time: 2025-06-13 17:11:34.785119
End Time: 2025-06-13 17:11:35.387118
Duration: 0.60 seconds
Steps Executed: 6
Tasks Executed: 11
Loops Executed: 2
Errors Encountered: 0

Playbook Metadata:
  playbook_path: ../../data/catalog/playbooks/weather_example.yaml


## Step Execution Analysis

Analyze the execution of steps in the playbook.


In [5]:
if execution_id:
    # step execution details
    step_results = pl.from_arrow(con.execute("""
        SELECT 
            step_id, 
            step_name, 
            parent_id, 
            timestamp, 
            status, 
            data, 
            error
        FROM step_results
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    step_results = parse_json_column(step_results, 'data')

    # step execution summary
    step_summary = pl.from_arrow(con.execute("""
        SELECT 
            step_name, 
            COUNT(*) as execution_count,
            COUNT(CASE WHEN status = 'success' THEN 1 END) as success_count,
            COUNT(CASE WHEN status = 'error' THEN 1 END) as error_count
        FROM step_results
        WHERE execution_id = ?
        GROUP BY step_name
        ORDER BY execution_count DESC
    """, [execution_id]).arrow())

    print("Step Execution Summary:")
    display(step_summary)

    # step results
    print("\nStep Results:")
    display(step_results)
else:
    print("No execution ID available.")


Step Execution Summary:


step_name,execution_count,success_count,error_count
str,i64,i64,i64
"""end_district_loop""",1,1,0
"""district_loop""",1,1,0
"""end_city_loop""",1,1,0
"""city_loop""",1,1,0
"""aggregate_alerts""",1,1,0
"""start""",1,1,0



Step Results:


step_id,step_name,parent_id,timestamp,status,data,error
str,str,str,datetime[μs],str,object,str
"""06f574ba-4d16-4966-b644-400d78…","""start""",null,2025-06-13 17:11:34.790691,"""success""",{},null
"""c5631b1a-82c5-4a2a-9a50-4d7204…","""city_loop""",null,2025-06-13 17:11:34.870774,"""success""","[{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}]",null
"""e8357f5e-bd58-4d26-9817-177dc8…","""district_loop""",null,2025-06-13 17:11:35.292841,"""success""","[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]",null
"""944ed098-3c32-491d-8c24-916fc3…","""end_district_loop""","""a95ed946-67de-40d4-b154-7608d7…",2025-06-13 17:11:35.363378,"""success""",{'district_loop_output': None},null
"""82fe9544-6f7d-43fb-a33a-e55c69…","""end_city_loop""","""d82d33e2-4931-4b05-9542-308205…",2025-06-13 17:11:35.377592,"""success""",{'alerts': None},null
"""4d3ae245-e6c9-4e07-b784-a96f1d…","""aggregate_alerts""",null,2025-06-13 17:11:35.384021,"""success""",{},null


## Loop Execution Analysis

Analyze the execution of loops in the playbook.


In [6]:
if execution_id:
    # loop execution details
    loop_state = pl.from_arrow(con.execute("""
        SELECT 
            loop_id, 
            loop_name, 
            parent_id, 
            iterator, 
            items, 
            current_index, 
            current_item, 
            results, 
            timestamp, 
            status
        FROM loop_state
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    loop_state = parse_json_column(loop_state, 'items')
    loop_state = parse_json_column(loop_state, 'current_item')
    loop_state = parse_json_column(loop_state, 'results')

    # loop execution summary
    if len(loop_state) > 0:
        print("Loop Execution Summary:")
        display(loop_state)

        # loop iteration events
        loop_events = pl.from_arrow(con.execute("""
            SELECT 
                event_id, 
                node_name, 
                event_type, 
                timestamp, 
                status, 
                duration, 
                output_result as data, 
                metadata
            FROM event_log
            WHERE execution_id = ? AND event_type LIKE '%loop%'
            ORDER BY timestamp
        """, [execution_id]).arrow())

        loop_events = parse_json_column(loop_events, 'data')
        loop_events = parse_json_column(loop_events, 'metadata')

        print("\nLoop Events:")
        display(loop_events)
    else:
        print("No loop executions found.")
else:
    print("No execution ID available.")


Loop Execution Summary:


loop_id,loop_name,parent_id,iterator,items,current_index,current_item,results,timestamp,status
str,str,str,str,object,i32,object,object,datetime[μs],str
"""a95ed946-67de-40d4-b154-7608d7…","""district_loop""","""e8357f5e-bd58-4d26-9817-177dc8…","""district""","[{'name': 'Downtown', 'population': 50000}, {'name': 'North', 'population': 25000}, {'name': 'East', 'population': 30000}, {'name': 'Mordor', 'population': 666}]",4,null,"[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]",2025-06-13 17:11:35.362710,"""completed"""
"""d82d33e2-4931-4b05-9542-308205…","""city_loop""","""c5631b1a-82c5-4a2a-9a50-4d7204…","""city""","[{'name': 'London', 'lat': 51.51, 'lon': -0.13}, {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}, {'name': 'Berlin', 'lat': 52.52, 'lon': 13.41}]",3,null,"[{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}]",2025-06-13 17:11:35.376774,"""completed"""



Loop Events:


event_id,node_name,event_type,timestamp,status,duration,data,metadata
str,str,str,datetime[μs],str,f64,object,object
"""8cbf1c97-4a66-4fa3-9619-b7649f…","""city_loop""","""loop_start""",2025-06-13 17:11:34.804876,"""in_progress""",0.0,null,"{'item_count': 3, 'iterator': 'city'}"
"""d44ef999-9f05-40ff-a1ca-e540f7…","""city_loop[0]""","""loop_iteration""",2025-06-13 17:11:34.809812,"""in_progress""",0.0,null,"{'index': 0, 'item': {'name': 'London', 'lat': 51.51, 'lon': -0.13}}"
"""8e6ec4cf-7397-4f68-ae7d-ea1fb0…","""city_loop[0]""","""loop_iteration_complete""",2025-06-13 17:11:34.829778,"""success""",0.0,"{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}","{'index': 0, 'item': {'name': 'London', 'lat': 51.51, 'lon': -0.13}}"
"""c1c4fc1f-4da7-4663-81b5-9f4102…","""city_loop[1]""","""loop_iteration""",2025-06-13 17:11:34.832019,"""in_progress""",0.0,null,"{'index': 1, 'item': {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}}"
"""2f616522-de32-460b-bcab-3f794e…","""city_loop[1]""","""loop_iteration_complete""",2025-06-13 17:11:34.847940,"""success""",0.0,"{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}","{'index': 1, 'item': {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}}"
…,…,…,…,…,…,…,…
"""186e5963-c739-451c-8836-a866bb…","""district_loop""","""loop_complete""",2025-06-13 17:11:35.298089,"""success""",0.380932,"[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]","{'item_count': 4, 'processed_count': 4}"
"""bdf8812d-436b-4a6d-9c3d-52356a…","""end_district_loop""","""end_loop_start""",2025-06-13 17:11:35.352810,"""in_progress""",0.0,null,{'loop_name': 'district_loop'}
"""bfd30142-4433-404c-8923-3bed6f…","""end_district_loop""","""end_loop_complete""",2025-06-13 17:11:35.364780,"""success""",0.011913,{'district_loop_output': None},{'loop_name': 'district_loop'}


## Task Execution Analysis

Analyze the execution of tasks in the playbook.


In [7]:
if execution_id:
    # task execution details
    task_results = pl.from_arrow(con.execute("""
        SELECT 
            task_id, 
            task_name, 
            task_type, 
            parent_id, 
            timestamp, 
            status, 
            data, 
            error
        FROM task_results
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    task_results = parse_json_column(task_results, 'data')

    # task execution summary
    task_summary = pl.from_arrow(con.execute("""
        SELECT 
            task_name, 
            task_type,
            COUNT(*) as execution_count,
            COUNT(CASE WHEN status = 'success' THEN 1 END) as success_count,
            COUNT(CASE WHEN status = 'error' THEN 1 END) as error_count
        FROM task_results
        WHERE execution_id = ?
        GROUP BY task_name, task_type
        ORDER BY execution_count DESC
    """, [execution_id]).arrow())

    print("Task Execution Summary:")
    display(task_summary)

    # task results
    print("\nTask Results:")
    display(task_results)
else:
    print("No execution ID available.")


Task Execution Summary:


task_name,task_type,execution_count,success_count,error_count
str,str,i64,i64,i64
"""process_district""","""python""",5,5,0
"""get_forecast""","""http""",4,4,0
"""alert_task""","""http""",1,1,0
"""get_city_districts""","""http""",1,1,0



Task Results:


task_id,task_name,task_type,parent_id,timestamp,status,data,error
str,str,str,str,datetime[μs],str,object,str
"""1e35ac82-f770-40a7-be8b-7f9fff…","""get_forecast""","""http""","""dfe28548-f673-4b63-afd7-1359bb…",2025-06-13 17:11:34.818595,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""59e7f8cc-b08b-44b2-b5fd-58ed37…","""get_forecast""","""http""","""071677d7-a593-445a-bf89-ea62ee…",2025-06-13 17:11:34.840619,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""a9dadad8-cfd8-4592-a536-7eca90…","""get_forecast""","""http""","""ca5920a7-05b9-4865-b368-c89fb7…",2025-06-13 17:11:34.860389,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""bb5c3bbc-e4d1-40a0-85be-839b9c…","""get_forecast""","""http""","""ff40a303-31a5-46cb-bbff-68f755…",2025-06-13 17:11:34.880884,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""b050f258-242d-4816-b3ca-2984bf…","""alert_task""","""http""","""4110d8ca-ef0d-4e9e-b245-adc984…",2025-06-13 17:11:34.894907,"""success""","{'data': 'mocked_response', 'status': 'success'}",null
…,…,…,…,…,…,…,…
"""8cf030da-664a-4ead-a512-1f51ff…","""process_district""","""python""","""24cac83c-f528-4791-b06d-caa2a0…",2025-06-13 17:11:34.976642,"""success""","{'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",null
"""03fe661e-1ce6-48bc-9989-74900c…","""process_district""","""python""","""2ff53f64-caea-44de-93f8-ff92e2…",2025-06-13 17:11:35.003420,"""success""","{'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",null
"""39aa9fd8-338c-444f-9c0b-3e493d…","""process_district""","""python""","""00c09c5e-72f0-4c5a-a134-d04be9…",2025-06-13 17:11:35.025802,"""success""","{'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",null


## Context Analysis

Analyze the context data during playbook execution.


In [8]:
if execution_id:
    # context data
    context_data = pl.from_arrow(con.execute("""
        SELECT 
            key, 
            value, 
            timestamp
        FROM context
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    context_data = parse_json_column(context_data, 'value')

    if len(context_data) > 0:
        print("Context Data:")
        display(context_data)

        # context state
        final_context = {}
        for _, row in context_data.to_pandas().iterrows():
            final_context[row['key']] = row.get('value')

        print("\nFinal Context State:")
        for key, value in final_context.items():
            print(f"\n{key}:")
            if isinstance(value, (dict, list)):
                print(json.dumps(value, indent=2))
            else:
                print(value)
    else:
        print("No context data found.")
else:
    print("No execution ID found.")


Context Data:


key,value,timestamp
str,object,datetime[μs]
"""workload""","{'jobId': '{{ job.uuid }}', 'state': 'ready', 'cities': [{'name': 'London', 'lat': 51.51, 'lon': -0.13}, {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}, {'name': 'Berlin', 'lat': 52.52, 'lon': 13.41}], 'base_url': 'https://api.open-meteo.com/v1', 'temperature_threshold': 25}",2025-06-13 17:11:34.782191
"""execution_start""",2025-06-13T17:11:34.784461,2025-06-13 17:11:34.784477
"""base_url""",https://api.open-meteo.com/v1,2025-06-13 17:11:34.854093
"""temperature_threshold""",25,2025-06-13 17:11:34.854788
"""city_loop_results""","[{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}]",2025-06-13 17:11:34.870053
…,…,…
"""district_loop_results""","[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]",2025-06-13 17:11:35.291415
"""process_district""","{'result': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}, 'status': 'success'}",2025-06-13 17:11:35.342145
"""result""","{'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",2025-06-13 17:11:35.346993



Final Context State:

workload:
{
  "jobId": "{{ job.uuid }}",
  "state": "ready",
  "cities": [
    {
      "name": "London",
      "lat": 51.51,
      "lon": -0.13
    },
    {
      "name": "Paris",
      "lat": 48.85,
      "lon": 2.35
    },
    {
      "name": "Berlin",
      "lat": 52.52,
      "lon": 13.41
    }
  ],
  "base_url": "https://api.open-meteo.com/v1",
  "temperature_threshold": 25
}

execution_start:
2025-06-13T17:11:34.784461

base_url:
https://api.open-meteo.com/v1

temperature_threshold:
25

city_loop_results:
[
  {
    "fetch_and_evaluate": {
      "data": {
        "hourly": {
          "temperature_2m": [
            20,
            22,
            25,
            28,
            30,
            26,
            24
          ],
          "precipitation_probability": [
            0,
            10,
            20,
            30,
            20,
            10,
            0
          ],
          "windspeed_10m": [
            10,
            12,
            

## Preview `event_log` table


In [9]:
table_name = 'event_log'
df = pl.from_arrow(con.execute(f"SELECT * FROM {table_name} LIMIT 20").arrow())
df


execution_id,event_id,parent_event_id,timestamp,event_type,node_id,node_name,node_type,status,duration,input_context,output_result,metadata
str,str,str,datetime[μs],str,str,str,str,str,f64,str,str,str
"""068ab870-2561-4c85-a44a-d6758c…","""8958c1b4-b7b4-4272-b8d7-ffc8ef…",null,2025-06-13 15:00:51.321943,"""execution_start""","""068ab870-2561-4c85-a44a-d6758c…","""weather_iterator_example""","""playbook""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""playbook_path"": ""../../data/…"
"""068ab870-2561-4c85-a44a-d6758c…","""a0e3a391-2b24-4ecb-9683-9c1425…",null,2025-06-13 15:00:51.322739,"""step_start""","""bd3933c8-65ef-4268-8e77-400e35…","""start""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
"""068ab870-2561-4c85-a44a-d6758c…","""3023e3a2-b108-4aeb-a8eb-11878a…","""a0e3a391-2b24-4ecb-9683-9c1425…",2025-06-13 15:00:51.323890,"""step_complete""","""bd3933c8-65ef-4268-8e77-400e35…","""start""","""step""","""success""",0.001293,"""{""jobId"": ""{{ job.uuid }}"", ""s…","""{}""","""{""step_type"": ""standard""}"""
"""068ab870-2561-4c85-a44a-d6758c…","""8b509bea-dcdb-4b77-b391-5e5029…","""8958c1b4-b7b4-4272-b8d7-ffc8ef…",2025-06-13 15:00:51.326760,"""step_transition""","""068ab870-2561-4c85-a44a-d6758c…","""transition_to_city_loop""","""transition""","""success""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""from_step"": ""start"", ""to_ste…"
"""068ab870-2561-4c85-a44a-d6758c…","""ff080df5-8ca3-426c-a5b7-53f7ff…",null,2025-06-13 15:00:51.327629,"""step_start""","""2bd77523-e60f-492d-9371-deac39…","""city_loop""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""068ab870-2561-4c85-a44a-d6758c…","""a3c71de3-57e8-47bb-974e-e16532…",null,2025-06-13 15:00:51.340895,"""step_start""","""daafcba8-8774-4bf0-b9e6-44ba3d…","""fetch_and_evaluate""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
"""068ab870-2561-4c85-a44a-d6758c…","""23e6b302-04e5-4fb4-a364-908a74…","""a3c71de3-57e8-47bb-974e-e16532…",2025-06-13 15:00:51.341562,"""task_execute""","""f638e9e1-869f-48a4-a3e7-f2dc0c…","""get_forecast""","""task.http""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""task_type"": ""http""}"""
"""068ab870-2561-4c85-a44a-d6758c…","""a6e987cd-eef7-4087-b2f5-d42302…","""23e6b302-04e5-4fb4-a364-908a74…",2025-06-13 15:00:51.342561,"""task_start""","""2d68bb1b-7dbc-429b-a75f-de5192…","""get_forecast""","""http""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""method"": ""GET"", ""endpoint"": …"


## Show all steps resluts


In [10]:
pl.from_arrow(con.execute("SELECT * FROM step_results ORDER BY timestamp DESC LIMIT 10").arrow())


execution_id,step_id,step_name,parent_id,timestamp,status,data,error
str,str,str,str,datetime[μs],str,str,str
"""da456514-00e8-4e99-bb18-02d50e…","""4d3ae245-e6c9-4e07-b784-a96f1d…","""aggregate_alerts""",null,2025-06-13 17:11:35.384021,"""success""","""{}""",null
"""da456514-00e8-4e99-bb18-02d50e…","""82fe9544-6f7d-43fb-a33a-e55c69…","""end_city_loop""","""d82d33e2-4931-4b05-9542-308205…",2025-06-13 17:11:35.377592,"""success""","""{""alerts"": null}""",null
"""da456514-00e8-4e99-bb18-02d50e…","""944ed098-3c32-491d-8c24-916fc3…","""end_district_loop""","""a95ed946-67de-40d4-b154-7608d7…",2025-06-13 17:11:35.363378,"""success""","""{""district_loop_output"": null}""",null
"""da456514-00e8-4e99-bb18-02d50e…","""e8357f5e-bd58-4d26-9817-177dc8…","""district_loop""",null,2025-06-13 17:11:35.292841,"""success""","""[{""process_district"": {""city"":…",null
"""da456514-00e8-4e99-bb18-02d50e…","""c5631b1a-82c5-4a2a-9a50-4d7204…","""city_loop""",null,2025-06-13 17:11:34.870774,"""success""","""[{""fetch_and_evaluate"": {""data…",null
"""da456514-00e8-4e99-bb18-02d50e…","""06f574ba-4d16-4966-b644-400d78…","""start""",null,2025-06-13 17:11:34.790691,"""success""","""{}""",null
"""e453e58d-c5a4-4109-8a55-dcf424…","""d99595ad-0e64-43fc-9fce-3dae5c…","""aggregate_alerts""",null,2025-06-13 17:10:35.736193,"""success""","""{}""",null
"""e453e58d-c5a4-4109-8a55-dcf424…","""0006ce60-af55-4c9e-aff1-acb8bc…","""end_city_loop""","""7cd01222-4663-4864-b183-335e0f…",2025-06-13 17:10:35.544099,"""success""","""{""alerts"": null}""",null
"""e453e58d-c5a4-4109-8a55-dcf424…","""9b3e34da-7c84-4da2-a43b-1b7171…","""end_district_loop""","""44ac87d6-5c9e-4c73-9bf3-a11d69…",2025-06-13 17:10:35.527097,"""success""","""{""district_loop_output"": null}""",null


## Show all task results

In [11]:
con.sql("SELECT * FROM task_results ORDER BY timestamp DESC LIMIT 10").show()

┌──────────────────────────────────────┬──────────────────────────────────────┬────────────────────┬───────────┬──────────────────────────────────────┬────────────────────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────┐
│             execution_id             │               task_id                │     task_name      │ task_type │              parent_id               │         timestamp          │ status  │                                                                                                     data                                                                                                      │  error  │
│               varchar                │               varchar                │      varchar       │  varchar  │               varchar                │         timestamp          │ v

In [12]:
%sql SELECT * FROM task_results ORDER BY timestamp DESC LIMIT 10

Running query in 'duckdb'

execution_id,task_id,task_name,task_type,parent_id,timestamp,status,data,error
da456514-00e8-4e99-bb18-02d50ea3ec86,c2f554d5-3e50-4897-9557-ab39640175e3,process_district,python,cb9d58fa-07dc-420d-965e-361d1660b2c7,2025-06-13 17:11:35.338094,success,"{""city"": ""Berlin"", ""district"": ""Mordor"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
da456514-00e8-4e99-bb18-02d50ea3ec86,acc5cc7f-a1f1-4c27-85e7-4a8c7a2ac3fa,process_district,python,19becf44-ecc7-4455-af2e-107b0472359d,2025-06-13 17:11:35.276267,success,"{""city"": ""Berlin"", ""district"": ""Mordor"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
da456514-00e8-4e99-bb18-02d50ea3ec86,39aa9fd8-338c-444f-9c0b-3e493df67758,process_district,python,00c09c5e-72f0-4c5a-a134-d04be9c574ae,2025-06-13 17:11:35.025802,success,"{""city"": ""Berlin"", ""district"": ""East"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
da456514-00e8-4e99-bb18-02d50ea3ec86,03fe661e-1ce6-48bc-9989-74900c090979,process_district,python,2ff53f64-caea-44de-93f8-ff92e2a4745c,2025-06-13 17:11:35.003420,success,"{""city"": ""Berlin"", ""district"": ""North"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
da456514-00e8-4e99-bb18-02d50ea3ec86,8cf030da-664a-4ead-a512-1f51ffa5784d,process_district,python,24cac83c-f528-4791-b06d-caa2a023c43d,2025-06-13 17:11:34.976642,success,"{""city"": ""Berlin"", ""district"": ""Downtown"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
da456514-00e8-4e99-bb18-02d50ea3ec86,4855198f-a887-4849-9a8c-4ff8ea097236,get_city_districts,http,6a5d8a40-f80d-4b12-8ea0-4059563de406,2025-06-13 17:11:34.905825,success,"{""data"": [{""name"": ""Downtown"", ""population"": 50000}, {""name"": ""North"", ""population"": 25000}, {""name"": ""East"", ""population"": 30000}, {""name"": ""Mordor"", ""population"": 666}]}",None
da456514-00e8-4e99-bb18-02d50ea3ec86,b050f258-242d-4816-b3ca-2984bfd327e2,alert_task,http,4110d8ca-ef0d-4e9e-b245-adc984d74ff1,2025-06-13 17:11:34.894907,success,"{""data"": ""mocked_response"", ""status"": ""success""}",None
da456514-00e8-4e99-bb18-02d50ea3ec86,bb5c3bbc-e4d1-40a0-85be-839b9c2b0605,get_forecast,http,ff40a303-31a5-46cb-bbff-68f755c30996,2025-06-13 17:11:34.880884,success,"{""data"": {""hourly"": {""temperature_2m"": [20, 22, 25, 28, 30, 26, 24], ""precipitation_probability"": [0, 10, 20, 30, 20, 10, 0], ""windspeed_10m"": [10, 12, 15, 18, 22, 19, 14]}}, ""alert"": true, ""max_temp"": 30}",None
da456514-00e8-4e99-bb18-02d50ea3ec86,a9dadad8-cfd8-4592-a536-7eca908a9e2a,get_forecast,http,ca5920a7-05b9-4865-b368-c89fb76ba1a8,2025-06-13 17:11:34.860389,success,"{""data"": {""hourly"": {""temperature_2m"": [20, 22, 25, 28, 30, 26, 24], ""precipitation_probability"": [0, 10, 20, 30, 20, 10, 0], ""windspeed_10m"": [10, 12, 15, 18, 22, 19, 14]}}, ""alert"": true, ""max_temp"": 30}",None
da456514-00e8-4e99-bb18-02d50ea3ec86,59e7f8cc-b08b-44b2-b5fd-58ed37970601,get_forecast,http,071677d7-a593-445a-bf89-ea62ee9871c0,2025-06-13 17:11:34.840619,success,"{""data"": {""hourly"": {""temperature_2m"": [20, 22, 25, 28, 30, 26, 24], ""precipitation_probability"": [0, 10, 20, 30, 20, 10, 0], ""windspeed_10m"": [10, 12, 15, 18, 22, 19, 14]}}, ""alert"": true, ""max_temp"": 30}",None
